<td>
   <a target="_blank" href="https://labelbox.com" ><img src="https://labelbox.com/blog/content/images/2021/02/logo-v4.svg" width=256/></a>
</td>

<td>
<a href="https://colab.research.google.com/github/Labelbox/labelbox-python/blob/develop/examples/annotation_import/video.ipynb" target="_blank"><img
src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>
</td>

<td>
<a href="https://github.com/Labelbox/labelbox-python/tree/develop/examples/annotation_import/video.ipynb" target="_blank"><img
src="https://img.shields.io/badge/GitHub-100000?logo=github&logoColor=white" alt="GitHub"></a>
</td>

# Video MAL

* Upload model inferences for video tasks
* Support types
    * bounding box

In [ ]:
!pip install -q labelbox

In [ ]:
import os
import uuid
from io import BytesIO
from typing import Dict, Any, Tuple

from labelbox import Client, LabelingFrontend
from labelbox.schema.ontology import OntologyBuilder, Tool, Classification, Option

# API Key and Client
Provide a valid api key below in order to properly connect to the Labelbox Client.

In [ ]:
# Add your api key
API_KEY = None
client = Client(api_key=API_KEY)

### Project Setup

In [ ]:
# We want to try out a few different tools here.
ontology_builder = OntologyBuilder(
    tools=[Tool(tool=Tool.Type.BBOX, name="jellyfish")])

In [ ]:
# Lets setup a project to label
# Note see Ontology, Project, and Project_setup notebooks for more information on this section.
project = client.create_project(name="video_mal_project")
dataset = client.create_dataset(name="video_mal_dataset")
dataset.create_data_row(
    row_data=
    "https://storage.labelbox.com/cjhfn5y6s0pk507024nz1ocys%2Fb8837f3b-b071-98d9-645e-2e2c0302393b-jellyfish2-100-110.mp4"
)
editor = next(
    client.get_labeling_frontends(where=LabelingFrontend.name == "Editor"))
project.setup(editor, ontology_builder.asdict())
project.datasets.connect(dataset)

#### Grab featureSchemaIds

In [ ]:
# When we created a project with the ontology defined above, all of the ids were assigned.
# So lets reconstruct the ontology builder with all of the ids.
ontology = ontology_builder.from_project(project)
# We want all of the feature schemas to be easily accessible by name.
schema_lookup = {tool.name: tool.feature_schema_id for tool in ontology.tools}
print(schema_lookup)

{'jellyfish': 'cky3dt2lja37d0z9t26wf3qo5'}


## Import Format

* [Documentation](https://docs.labelbox.com/docs/bounding-box-json)


```
Each row of the import is a unique instance

schemaId: <featureSchemaId>
dataRow:
    id: <dataRowId>
Instance:
    [Segments]:
        [KeyFrames]:
            frame:
            bbox:
                top:
                bottom:
                height:
                width:
```

**segments**: A segment represents a continuous section where an object is visible. If an instance disappears then the segment ends. If it re-appears, a new segment is created.

**keyframes**: Key frames identify the location of an instance. Between keyframes, the location of the instance is interpolated.

**bbox**: The coordinates of the bounding box

In [ ]:
segments = [{
    "keyframes": [{
        "frame": 1,
        "bbox": {
            "top": 80,
            "left": 80,
            "height": 80,
            "width": 80
        }
    }, {
        "frame": 20,
        "bbox": {
            "top": 125,
            "left": 125,
            "height": 200,
            "width": 300
        }
    }]
}, {
    "keyframes": [{
        "frame": 27,
        "bbox": {
            "top": 80,
            "left": 50,
            "height": 80,
            "width": 50
        }
    }]
}]

##### Create helper functions to make this much easier

In [ ]:
def create_video_bbox_ndjson(datarow_id: str, schema_id: str,
                             segments: Dict[str, Any]) -> Dict[str, Any]:
    return {
        "uuid": str(uuid.uuid4()),
        "schemaId": schema_id,
        "dataRow": {
            "id": datarow_id
        },
        "segments": segments
    }

In [ ]:
uploads = []

for data_row in dataset.data_rows():
    uploads.append(
        create_video_bbox_ndjson(data_row.uid, schema_lookup['jellyfish'],
                                 segments))

### Upload the annotations

In [ ]:
# Let's upload!
# Validate must be set to false for video bounding boxes
upload_task = project.upload_annotations(name=f"upload-job-{uuid.uuid4()}",
                                         annotations=uploads,
                                         validate=False)

In [ ]:
# Wait for upload to finish (Will take up to five minutes)
upload_task.wait_until_done()
# Review the upload status
print(upload_task.errors)

[]
